In [1]:
import torch
import re
import os
import os.path
import json
import pickle
import openai
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import random
from dotenv import load_dotenv

### Dotenv

In [2]:
load_dotenv()
openai.api_key = os.environ['OPENAI_API']

### DataFrames

In [3]:
root = "../../../data/OpenAI/DataFrames/"

file = "DF_Andrei.csv"
df_Andrei = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Andrei_species = list(df_Andrei.index)


file = "DF_Daniel.csv"
df_Daniel = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')\
    .dropna()
df_Daniel_species = list(df_Daniel.index)

file = "DF_Pierre.csv"
df_Pierre = pd.read_csv(F"{root}{file}", header=[0, 1], index_col=0)\
    .rename_axis('Species', axis='index')
df_Pierre_species = list(df_Pierre.index)

### Trait Dicts

In [4]:
folder_traits = "../../../data/OpenAI/Traits/"

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

with open(F"{folder_traits}Andrei.json", 'r') as f:
  caribbean_traits_dict = json.load(f)

### Functions

In [5]:
def combine_words_with_capital(string):
    # remove non-alphanumeric characters
    string = re.sub(r'[^\w\s/]', '', string)
    # split the string on the slash ("/")
    parts = string.split('/')
    # combine words with capitalization for each part
    parts = [''.join(word.capitalize() for word in part.split()) for part in parts]
    # join the parts with an empty string
    return ''.join(parts)

def create_clean_paragraphs(input_dict):

    # Create a new dictionary to store the cleaned-up values
    output_dict = {}
    
    # Loop through the keys and values of the input dictionary
    for key, value in input_dict.items():
        # Convert the list of values to a set to remove duplicates
        unique_values = set(value)
        
        # Join the sentences together into a single string
        combined_string = ' '.join(unique_values)
        
        # Add the cleaned-up string to the output dictionary
        output_dict[key] = combined_string
    
    # Return the cleaned-up dictionary
    return output_dict

### Text Data
#### Caribbean

In [6]:
folder_sents = "../../../data/OpenAI/DescriptionSnippets/SentencesOriginal/"

caribbean_file = "Sents_Caribbean.pkl"
caribbean_sents = pickle.load(open(F"{folder_sents}{caribbean_file}", 'rb'))

In [7]:
caribbean_paras = create_clean_paragraphs(caribbean_sents)

In [8]:
test_list = ['Avicennia germinans',
 'Metopium brownei',
 'Handroanthus billbergii',
 'Bourreria succulenta',
 'Bursera karsteniana',
 'Bursera simaruba',
 'Bursera tomentosa',
 'Cynophalla flexuosa',
 'Cynophalla hastata',
 'Quadrella odoratissima',
 'Crossopetalum rhacoma',
 'Maytenus versluysii',
 'Clusia rosea',
 'Conocarpus erectus',
 'Laguncularia racemosa']

In [9]:
# Define the path to the directory where the prompts and results will be saved
folder_prompts = "../../../data/OpenAI/PromptsResults/OriginalData/"

choices = [
    1, 0, 
]

# Loop over each JSON file in the list caribbean_jsons_paras
for idx, (species, text) in enumerate(caribbean_paras.items()):

    if species not in test_list:
        continue
    
    # Replace spaces in the species name with underscores
    folder_species = species.replace(' ', '_')

    # Try to create a directory for the prompts for the species
    try:
        os.makedirs(F"{folder_prompts}{folder_species}")
    except FileExistsError:
        pass

    # Loop over each trait and trait options in the caribbean_traits_dict dictionary
    for trait, trait_options in (pbar := tqdm(caribbean_traits_dict.items(), leave=False, position=0)):
        pbar.set_description(f"{idx}: {species}")

        # Create the question and options for the ChatGPT prompt
        question = F"Which of the following values correctly describe(s) the '{trait}' trait mentioned in the text? Fill in a '1', if the trait value is likely to occur based on the text, '0' if it is unlikely to occur and 'NA' if there is no information that can be used to infer the occurrence of the trait."
        format = F"\nPlease ONLY return a Python list of tuples with ALL the options/value combinations from the list with options, like this: [({trait_options[0]}, {random.choice(choices)}), (({trait_options[1]}, {random.choice(choices)}))]"
        options = F"\nThe possible values for this trait are: {trait_options}."\

        user_content = F"{question} {options} {format}"
        # print(user_content)

        # Combine the words in the trait name with capital letters and use this as the file name
        file_name = combine_words_with_capital(trait)
        # Check if file is already there (OpenAI Outage)
        if os.path.exists(F"{folder_prompts}{folder_species}/{file_name}.json"):
            continue

        # Create the messages to send to the ChatGPT API
        messages = [
            {"role": "assistant", "content": text},
            {"role": "user", "content": user_content}
            ]
        # Call the ChatGPT API to generate a completion for the prompt
        completion = openai.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = messages,
        )


        # Save the completion to a JSON file with the file name in the species directory
        with open(F"{folder_prompts}{folder_species}/{file_name}.json", 'w') as fp:
            json.dump(completion, fp)
            


### Testing

In [ ]:
species

In [ ]:
text

In [ ]:
caribbean_traits_dict

trait = list(caribbean_traits_dict.keys())[22]
trait_options = caribbean_traits_dict[trait]

In [ ]:
trait

In [ ]:
trait_options

In [ ]:
choices = [
    1, 0, 
]

question = F"Which of the following values correctly describe(s) the '{trait}' trait mentioned in the text? Fill in a '1', if the trait value is likely to occur based on the text, '0' if it is unlikely to occur and 'NA' if there is no information that can be used to infer the occurrence of the trait."
format = F"\nPlease ONLY return a Python list of tuples with ALL the options/value combinations from the list with options, like this: [('{trait_options[0]}', '{random.choice(choices)}'), ('{trait_options[1]}', '{random.choice(choices)}')] or in case there is not information: [('{trait_options[0]}', 'NA'), ('{trait_options[1]}', 'NA')]"
options = F"\nThe possible values for this trait are: {trait_options}."\

user_content = F"{question} {options} {format}"

In [ ]:
print(text)

In [ ]:
print(F"""text:
{text}
""")
print(user_content)

In [ ]:
# Create the messages to send to the ChatGPT API
messages = [
    {"role": "assistant", "content": text},
    {"role": "user", "content": user_content}
    ]
# Call the ChatGPT API to generate a completion for the prompt
completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = messages,
)

In [ ]:
completion